In [8]:
import pandas as pd

df = pd.read_csv("data_raw")
df.shape

FileNotFoundError: [Errno 2] No such file or directory: 'data_raw'

In [89]:
df.columns

Index(['AGE', 'CLAIM_ID', 'CLAIM_NUMBER', 'CLAIMANT_TYPE_DESC', 'GENDER',
       'AGE_GROUP', 'NATURE_OF_INJURY_DESC', 'BODY_PART_DESC',
       'BODY_PART_GROUP_DESC', 'INCIDENT_STATE', 'CLAIM_CAUSE_GROUP_DESC',
       'medical_amount', 'cost_trend_strength', 'cost_volatility',
       'avg_daily_cost_change', 'inflation_adjusted_amount', 'icd_code_comb',
       'unique_icd_codes_count', 'icd_codes_per_visit',
       'MEDICAL_PAYMENT_TOTAL', 'NO_OF_VISIT', 'source'],
      dtype='object')

In [90]:
df.head()

,AGE,CLAIM_ID,CLAIM_NUMBER,CLAIMANT_TYPE_DESC,GENDER,AGE_GROUP,NATURE_OF_INJURY_DESC,BODY_PART_DESC,BODY_PART_GROUP_DESC,INCIDENT_STATE,...,cost_trend_strength,cost_volatility,avg_daily_cost_change,inflation_adjusted_amount,icd_code_comb,unique_icd_codes_count,icd_codes_per_visit,MEDICAL_PAYMENT_TOTAL,NO_OF_VISIT,source
0,58.0,SSUIPQPMC4,IC6FWZ6L7X,Future Medical III,Male,Senior_Citizen,SPRAIN,BACK: NOC,TRUNK,CA,...,0,0.0,0,30.00,847.2;847.1,2,0.04607,30.00,1,DATASET_NEW_4.csv
1,58.0,SSUIPQPMC4,IC6FWZ6L7X,Future Medical III,Male,Senior_Citizen,SPRAIN,BACK: NOC,TRUNK,CA,...,0,0.0,0,37.50,NaN,0,0.04607,30.00,2,DATASET_NEW_4.csv
2,58.0,SSUIPQPMC4,IC6FWZ6L7X,Future Medical III,Male,Senior_Citizen,SPRAIN,BACK: NOC,TRUNK,CA,...,0,0.0,0,45.35,NaN,0,0.04607,30.00,3,DATASET_NEW_4.csv
3,58.0,SSUIPQPMC4,IC6FWZ6L7X,Future Medical III,Male,Senior_Citizen,SPRAIN,BACK: NOC,TRUNK,CA,...,0,0.0,0,54.05,NaN,0,0.04607,38.70,4,DATASET_NEW_4.csv
4,58.0,SSUIPQPMC4,IC6FWZ6L7X,Future Medical III,Male,Senior_Citizen,SPRAIN,BACK: NOC,TRUNK,CA,...,0,0.0,0,56.27,NaN,0,0.04607,40.92,5,DATASET_NEW_4.csv


### Dataset overview
- Source: cleaned claims dataset
- Granularity: visit-level (multiple rows per claim)
- Rows: 933831
- Columns: 22

## Phase 1: Data Understanding

In [91]:
df.dtypes

AGE                          float64
CLAIM_ID                      object
CLAIM_NUMBER                  object
CLAIMANT_TYPE_DESC            object
GENDER                        object
AGE_GROUP                     object
NATURE_OF_INJURY_DESC         object
BODY_PART_DESC                object
BODY_PART_GROUP_DESC          object
INCIDENT_STATE                object
CLAIM_CAUSE_GROUP_DESC        object
medical_amount               float64
cost_trend_strength            int64
cost_volatility              float64
avg_daily_cost_change          int64
inflation_adjusted_amount    float64
icd_code_comb                 object
unique_icd_codes_count         int64
icd_codes_per_visit          float64
MEDICAL_PAYMENT_TOTAL        float64
NO_OF_VISIT                    int64
source                        object
dtype: object

In [92]:
missing_pct = df.isna().mean().sort_values(ascending=False) * 100
missing_pct

icd_code_comb                91.405833
AGE                          22.490686
AGE_GROUP                    22.490686
CLAIM_CAUSE_GROUP_DESC        0.239122
BODY_PART_DESC                0.011137
BODY_PART_GROUP_DESC          0.011137
INCIDENT_STATE                0.002249
NATURE_OF_INJURY_DESC         0.002142
CLAIM_NUMBER                  0.000000
CLAIM_ID                      0.000000
GENDER                        0.000000
CLAIMANT_TYPE_DESC            0.000000
cost_trend_strength           0.000000
medical_amount                0.000000
cost_volatility               0.000000
avg_daily_cost_change         0.000000
inflation_adjusted_amount     0.000000
unique_icd_codes_count        0.000000
icd_codes_per_visit           0.000000
MEDICAL_PAYMENT_TOTAL         0.000000
NO_OF_VISIT                   0.000000
source                        0.000000
dtype: float64

In [93]:
df[["AGE", "AGE_GROUP"]].head(10)

,AGE,AGE_GROUP
0,58.0,Senior_Citizen
1,58.0,Senior_Citizen
2,58.0,Senior_Citizen
3,58.0,Senior_Citizen
4,58.0,Senior_Citizen
5,58.0,Senior_Citizen
6,58.0,Senior_Citizen
7,58.0,Senior_Citizen
8,58.0,Senior_Citizen
9,58.0,Senior_Citizen


In [94]:
df["AGE"].isna().mean() * 100, df["AGE_GROUP"].isna().mean() * 100

(np.float64(22.490686216242555), np.float64(22.490686216242555))

### Age/age group and icd code handling
- missing values replaced with 'missing' + indicator for ICD code 
- 'age' column to be filled using median value as its more robust than mean 
- 'age group' column to be dropped after 'age' missing values have been filled

In [95]:
df["icd_missing"] = df["icd_code_comb"].isna().astype(int)

df["icd_code_comb_filled"] = df["icd_code_comb"].fillna("MISSING")

In [96]:
df["icd_missing"].value_counts(normalize=True)
df["icd_code_comb_filled"].value_counts().head()

icd_code_comb_filled
MISSING    853576
8472         2792
7998         2088
959.9        1916
959          1875
Name: count, dtype: int64

In [97]:
age_median = df["AGE"].median()
df["AGE"] = df["AGE"].fillna(age_median)

In [98]:
df["AGE"].isna().sum()

np.int64(0)

In [99]:
df["AGE"].describe()

count    933831.000000
mean         68.541366
std          10.389065
min          18.000000
25%          64.000000
50%          69.000000
75%          74.000000
max         125.000000
Name: AGE, dtype: float64

In [100]:
df.loc[df["AGE"] == age_median].head()

,AGE,CLAIM_ID,CLAIM_NUMBER,CLAIMANT_TYPE_DESC,GENDER,AGE_GROUP,NATURE_OF_INJURY_DESC,BODY_PART_DESC,BODY_PART_GROUP_DESC,INCIDENT_STATE,...,avg_daily_cost_change,inflation_adjusted_amount,icd_code_comb,unique_icd_codes_count,icd_codes_per_visit,MEDICAL_PAYMENT_TOTAL,NO_OF_VISIT,source,icd_missing,icd_code_comb_filled
559,69.0,PANFQC7LXV,WBV3GVSQFC,TD,Male,NaN,SPRAIN,MULTIPLE BODY PARTS,MULTIPLE BODY PARTS,CA,...,0,0.00,NaN,0,0.0,0.00,1,DATASET_NEW_4.csv,1,MISSING
560,69.0,PANFQC7LXV,WBV3GVSQFC,TD,Male,NaN,SPRAIN,MULTIPLE BODY PARTS,MULTIPLE BODY PARTS,CA,...,0,7.30,NaN,0,0.0,0.00,2,DATASET_NEW_4.csv,1,MISSING
561,69.0,PANFQC7LXV,WBV3GVSQFC,TD,Male,NaN,SPRAIN,MULTIPLE BODY PARTS,MULTIPLE BODY PARTS,CA,...,0,87.38,NaN,0,0.0,80.08,3,DATASET_NEW_4.csv,1,MISSING
562,69.0,PANFQC7LXV,WBV3GVSQFC,TD,Male,NaN,SPRAIN,MULTIPLE BODY PARTS,MULTIPLE BODY PARTS,CA,...,0,412.10,NaN,0,0.0,404.80,4,DATASET_NEW_4.csv,1,MISSING
563,69.0,PANFQC7LXV,WBV3GVSQFC,TD,Male,NaN,SPRAIN,MULTIPLE BODY PARTS,MULTIPLE BODY PARTS,CA,...,0,677.30,NaN,0,0.0,670.00,5,DATASET_NEW_4.csv,1,MISSING


In [101]:
df = df.drop(columns=["AGE_GROUP"])

In [102]:
"AGE_GROUP" in df.columns

False

## Cleaned data Baseline regression 
- predicting payment totals based on variable that do NOT directly construct it

In [103]:
y = df["MEDICAL_PAYMENT_TOTAL"]

In [104]:
leakage_cols = [
    "MEDICAL_PAYMENT_TOTAL",
    "medical_amount",
    "inflation_adjusted_amount",
    "CLAIM_ID",
    "CLAIM_NUMBER",
    "source"
]

X = df.drop(columns=leakage_cols)

In [105]:
categorical_cols = X.select_dtypes(include="object").columns.tolist()
numeric_cols = X.select_dtypes(exclude="object").columns.tolist()

In [106]:
categorical_cols, numeric_cols

(['CLAIMANT_TYPE_DESC',
  'GENDER',
  'NATURE_OF_INJURY_DESC',
  'BODY_PART_DESC',
  'BODY_PART_GROUP_DESC',
  'INCIDENT_STATE',
  'CLAIM_CAUSE_GROUP_DESC',
  'icd_code_comb',
  'icd_code_comb_filled'],
 ['AGE',
  'cost_trend_strength',
  'cost_volatility',
  'avg_daily_cost_change',
  'unique_icd_codes_count',
  'icd_codes_per_visit',
  'NO_OF_VISIT',
  'icd_missing'])

In [107]:
X[categorical_cols].nunique().sort_values(ascending=False)

icd_code_comb_filled      13451
icd_code_comb             13450
BODY_PART_DESC              108
NATURE_OF_INJURY_DESC        91
INCIDENT_STATE               38
CLAIMANT_TYPE_DESC           13
CLAIM_CAUSE_GROUP_DESC       10
BODY_PART_GROUP_DESC          8
GENDER                        2
dtype: int64

In [108]:
y = df["MEDICAL_PAYMENT_TOTAL"]

leakage_cols = [
    "MEDICAL_PAYMENT_TOTAL",
    "medical_amount",
    "inflation_adjusted_amount",
    "CLAIM_ID",
    "CLAIM_NUMBER",
    "source"
]

X = df.drop(columns=leakage_cols).copy()

In [109]:
X["icd_code_comb"] = X["icd_code_comb"].fillna("MISSING")
X["icd_missing"] = (X["icd_code_comb"] == "MISSING").astype(int)

In [110]:
icd_freq = X["icd_code_comb"].value_counts(normalize=True)
X["icd_code_freq"] = X["icd_code_comb"].map(icd_freq)

In [111]:
X = X.drop(columns=["icd_code_comb"])

In [112]:
X["AGE"] = X["AGE"].fillna(X["AGE"].median())
if "AGE_GROUP" in X.columns:
    X = X.drop(columns=["AGE_GROUP"])

In [113]:
categorical_cols = X.select_dtypes(include="object").columns.tolist()
categorical_cols

['CLAIMANT_TYPE_DESC',
 'GENDER',
 'NATURE_OF_INJURY_DESC',
 'BODY_PART_DESC',
 'BODY_PART_GROUP_DESC',
 'INCIDENT_STATE',
 'CLAIM_CAUSE_GROUP_DESC',
 'icd_code_comb_filled']

In [114]:
categorical_cols = [c for c in categorical_cols if c not in ["icd_code_comb", "icd_code_comb_filled"]]

In [115]:
icd_col = "icd_code_comb_filled"

In [116]:
X["icd_missing"] = (X[icd_col].isna() | (X[icd_col] == "MISSING")).astype(int)

X[icd_col] = X[icd_col].fillna("MISSING")
icd_freq = X[icd_col].value_counts(normalize=True)
X["icd_code_freq"] = X[icd_col].map(icd_freq)

X = X.drop(columns=[icd_col])

In [117]:
X.select_dtypes(include="object").nunique().sort_values(ascending=False).head(10)

BODY_PART_DESC            108
NATURE_OF_INJURY_DESC      91
INCIDENT_STATE             38
CLAIMANT_TYPE_DESC         13
CLAIM_CAUSE_GROUP_DESC     10
BODY_PART_GROUP_DESC        8
GENDER                      2
dtype: int64

In [118]:
X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)
X_encoded.shape

(933831, 272)

In [119]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

In [120]:
y = df["MEDICAL_PAYMENT_TOTAL"]

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42
)

In [121]:
lr = LinearRegression()
lr.fit(X_train, y_train)

,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"tol tol: float, default=1e-6The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for the `lsqr` solver.`tol` is set as `atol` and `btol` of :func:`scipy.sparse.linalg.lsqr` whenfitting on sparse training data. This parameter has no effect when fittingon dense data... versionadded:: 1.7",1e-06
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation. This will only providespeedup in case of sufficiently large problems, that is if firstly`n_targets > 1` and secondly `X` is sparse or if `positive` is setto `True`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",None
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive. Thisoption is only supported for dense arrays.For a comparison between a linear regression model with positive constraintson the regression coefficients and a linear regression without such constraints,see :ref:`sphx_glr_auto_examples_linear_model_plot_nnls.py`... versionadded:: 0.24",False


In [122]:
y_pred = lr.predict(X_test)

In [123]:
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

mae, rmse, r2

(4043.0263538448357, np.float64(11129.349869556803), 0.6969822770590995)

## Linear Regression analysis

- On average, based on the baseline linear regression model, we are off by approximately $4,4043. 
- The RMSE indicates that the model typically handles claims well, however, some large claims still drive error.
- The model explains 70% of the variance in total claim payments (SUBJECT TO CHANGE (seems too good)). 
  The 70% result is after reducing leakage areas, with human/medical processes, missing ICD structure, and heavy-tailed outcomes.
  So, conclusively, most of the variation in claim costs is explained by injury characteristics, visit patterns, and cost dynamics
  captured in the feature.  

In [124]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [125]:
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import numpy as np
import pandas as pd

In [1]:
import pandas as pd

In [3]:
claim_totals = (
    df
    .groupby("CLAIM_ID", as_index=False)
    .agg(
        TOTAL_MEDICAL_AMOUNT=("MEDICAL_AMOUNT", "sum"),
        TOTAL_VISITS=("NO_OF_VISIT", "max")
    )
)

NameError: name 'df' is not defined